# **<u>Proyecto de automatización de trabajo: iteración sobre Sheets en carpetas de Google Drive, y append en otra Sheets de BBDD consolidada.</u>**

---

## <span style="background-color: #FFC0CB">Pending:</span>
            1. establecer un trigger semanal
                      
          

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importación de bibliotecas y montaje de drive

In [2]:
import os
import time
import pandas as pd
from google.colab import drive
import gspread
from gspread_dataframe import set_with_dataframe  # Agrega esta línea
from oauth2client.service_account import ServiceAccountCredentials
from oauth2client.client import GoogleCredentials
from datetime import datetime

In [3]:
pip install pytz

In [4]:
import pytz

## Tranformar la zona horaria a Madrid con la biblioteca pytz


In [5]:
madrid_tz = pytz.timezone('Europe/Madrid')

# Obtiene la hora actual en la zona horaria de Madrid
madrid_time = datetime.now(madrid_tz)

print("Hora actual en Madrid:", madrid_time)

Hora actual en Madrid: 2024-10-22 16:44:51.895938+02:00


## Inicialización de credenciales de Google, Google API y archivo JSON

### **Atención**: previamente se debe crear un proyecto en la nube y seguir una serie de pasos para autenticar el acceso a sus archivos de drive. Más información [aquí](https://cloud.google.com/gcp/?hl=es&utm_source=bing&utm_medium=cpc&utm_campaign=emea-es-all-es-bkws-all-all-trial-e-gcp-1011340&utm_content=text-ad-none-any-DEV_c-CRE_-ADGP_Hybrid+%7C+BKWS+-+EXA+%7C+Txt+-+GCP+-+General+-+v3-KWID_43700061311461132-kwd-77240890777560:loc-170-userloc_164782&utm_term=KW_google%20cloud-NET_o-PLAC_&&refclickid=NNE_HotelDigitalMediaCampaign&pmed=DPM_CVG_TRV_HOT_NNE_SEM_GOG_&gclid=e13bffff37a5160b7e7e99c1ad5ecd04&gclsrc=3p.ds&+)


In [6]:
# Definir el ámbito de actuación y credenciales
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP (automat)/Automatización_py/test-automation-gsheets-5fa673850271.json', scope)

# Autenticarse con Google Sheets API
client = gspread.authorize(creds)

### Pre-procesamiento de datos:

* Nulos
* Columnas a descartar

In [7]:
# Acceder a la hoja de cálculo
sheet = client.open_by_key('1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8')

def obtener_estadisticas(worksheet):
    records = worksheet.get_all_records()
    num_filas = len(records)

    if num_filas == 0:
        print(f"\nHoja: {worksheet.title} - Vacía")
        return

    # Extraer valores numéricos de todas las columnas
    valores_numericos = [value for row in records for value in row.values() if isinstance(value, (int, float))]

    if valores_numericos:
        maximo = max(valores_numericos)
        minimo = min(valores_numericos)
    else:
        maximo = minimo = "No hay datos numéricos"

    celdas_totales = num_filas * len(records[0]) if records else 0
    celdas_vacias = sum([list(row.values()).count('') for row in records])

    print(f"\nHoja: {worksheet.title}")
    print(f"Máximo: {maximo}, Mínimo: {minimo}")
    print(f"Celdas vacías: {celdas_vacias} de {celdas_totales} celdas totales")

# Bucle para recorrer hojas 0, 1 y 2
for i in range(3):  # Hojas de índice 0, 1 y 2
    worksheet = sheet.get_worksheet(i)
    obtener_estadisticas(worksheet)



Hoja: Boletines autonómicos
Máximo: 6557, Mínimo: 1
Celdas vacías: 24724 de 97790 celdas totales

Hoja: Boletines parlamentarios
Máximo: 13421, Mínimo: 1
Celdas vacías: 45013 de 150168 celdas totales

Hoja: Agendas parlamentarias
Máximo: 2798, Mínimo: 1
Celdas vacías: 9985 de 37646 celdas totales


##Proceso de iteración por la carpeta de <u>**boletines parlamentarios**</u>, para identificar y levantar solamente los archivos que hayan sido modificados o subidos después de cierta fecha. Luego, los inserta debajo de la hoja correspondiente de base de datos unificada

### **  En el desarrollo, también incluyo otras operaciones de limpieza y transformación (los guión bajo de las fórmulas Excel, la modificación de las fechas, adaptación del timezone a Madrid, etc.)

In [8]:
from googleapiclient.discovery import build

# Credenciales y parámetros
sheet_name = 'Boletines_unificado'
sheet_id_unificado = '1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8'
CWD = '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP (automat)/Boletines Parlamentarios'
cutoff_date = pytz.timezone('Europe/Madrid').localize(datetime(2024, 10, 22, 12, 0, 0))  # Fecha de corte
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP (automat)/Automatización_py/test-automation-gsheets-5fa673850271.json', scope)
client = gspread.authorize(creds)

def limpiar_dataframe(dfp):
    """Limpia el DataFrame y convierte las fechas a cadenas con formato DD/MM/YYYY."""
    columns_to_replace = ['Sector', 'Marco geográfico', 'Proponente']
    for column in columns_to_replace:
        dfp[column] = dfp[column].astype(str).str.replace('_', ' ', regex=False)  # Reemplazar guiones bajos con espacios

    # Convertir las fechas a cadenas en formato DD/MM/YYYY
    dfp['Fecha publicación'] = pd.to_datetime(dfp['Fecha publicación'], errors='coerce').dt.strftime('%d/%m/%Y')

    # Rellenar valores nulos y convertir a lista de listas
    dfp_cleaned = dfp.fillna('')  # Rellenar los NaN con cadenas vacías
    data_cleaned = dfp_cleaned.values.tolist()  # Convertir a lista de listas
    return data_cleaned

def find_empty_row(worksheet):
    """Encuentra la primera fila vacía en la hoja"""
    str_list = list(filter(None, worksheet.col_values(1)))  # Obtiene la columna 1 y filtra filas vacías
    return len(str_list) + 1  # Devuelve el índice de la primera fila vacía

def obtener_sheet_id(sheet, worksheet_name):
    """Obtiene el sheetId de una hoja específica dentro del archivo de Google Sheets"""
    sheets_metadata = sheet.fetch_sheet_metadata()
    sheets = sheets_metadata['sheets']

    for s in sheets:
        if s['properties']['title'] == worksheet_name:
            return s['properties']['sheetId']

    raise Exception(f"No se encontró una hoja con el nombre: {worksheet_name}")

def aplicar_formato_fecha(sheet_id, worksheet_index, rango):
    """Aplica el formato de fecha DD/MM/YYYY en la columna J"""
    service = build('sheets', 'v4', credentials=creds)

    requests = [
        {
            "repeatCell": {
                "range": {
                    "sheetId": worksheet_index,  # Aquí se usa el ID correcto de la hoja
                    "startRowIndex": rango[0],  # Índice de fila de inicio (0-indexed)
                    "endRowIndex": rango[1],    # Índice de fila de fin (0-indexed)
                    "startColumnIndex": 9,  # Columna J (índice 9 basado en 0)
                    "endColumnIndex": 10   # Columna J termina en 10 (0-indexed)
                },
                "cell": {
                    "userEnteredFormat": {
                        "numberFormat": {
                            "type": "DATE",
                            "pattern": "DD/MM/YYYY"
                        }
                    }
                },
                "fields": "userEnteredFormat.numberFormat"
            }
        }
    ]

    body = {
        'requests': requests
    }

    # Ejecutar la solicitud batchUpdate para aplicar el formato de fecha
    service.spreadsheets().batchUpdate(spreadsheetId=sheet_id, body=body).execute()

def main():
    files = os.listdir(CWD)
    dataframes = []

    for file_ in files:
        file_path = os.path.join(CWD, file_)

        if os.path.isfile(file_path) and file_.endswith('.xlsx'):
            # Obtener la fecha de modificación del archivo
            file_mod_time = datetime.fromtimestamp(os.path.getmtime(file_path), pytz.utc).astimezone(pytz.timezone('Europe/Madrid'))

            # Procesar solo los archivos modificados después de la fecha de corte
            if file_mod_time > cutoff_date:
                dfp = pd.read_excel(file_path, engine='openpyxl')
                dataframes.append(dfp)
                print("DataFrame creado para el archivo:", file_)
                print(dfp)

                # Limpiar el DataFrame
                data_cleaned = limpiar_dataframe(dfp)

                # Obtener el cliente de Google Sheets y abrir el archivo unificado
                client = gspread.authorize(creds)
                sheet = client.open_by_key(sheet_id_unificado)

                # Obtener la segunda hoja del archivo
                worksheet = sheet.get_worksheet(1)  # Índice 1 es la segunda hoja
                sheet_id = obtener_sheet_id(sheet, worksheet.title)

                # Obtener el índice de la fila donde deseas insertar los datos
                indice_fila_insercion = find_empty_row(worksheet)
                print("Índice de fila de inserción:", indice_fila_insercion)

                # Insertar los datos limpios en la hoja seleccionada usando value_input_option='USER_ENTERED'
                worksheet.insert_rows(data_cleaned, indice_fila_insercion, value_input_option='USER_ENTERED')

                print("Datos insertados correctamente en la fila", indice_fila_insercion)

                # Aplicar formato de fecha a la columna J
                start_row = indice_fila_insercion - 1  # Convertir a índice basado en 0
                end_row = start_row + len(data_cleaned)
                aplicar_formato_fecha(sheet_id_unificado, sheet_id, (start_row, end_row, 9, 10))  # Formato en la columna J (columna 9)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


DataFrame creado para el archivo: 241022 Boletines Parlamentarios.xlsx
        ID          Sector Sector2  Subsector  Tema Legislatura  \
0    13555           Salud     NaN        NaN   NaN          XV   
1    13556           Salud     NaN        NaN   NaN          XV   
2    13557           Salud     NaN        NaN   NaN          XV   
3    13558        Primario     NaN        NaN   NaN          XV   
4    13559         Energía     NaN        NaN   NaN          XV   
..     ...             ...     ...        ...   ...         ...   
187  13742           Salud     NaN        NaN   NaN          XI   
188  13743           Salud     NaN        NaN   NaN          XI   
189  13744  Digitalización     NaN        NaN   NaN          XI   
190  13745       Movilidad     NaN        NaN   NaN        XIII   
191  13746       Movilidad     NaN        NaN   NaN        XIII   

    Marco geográfico                            Título de la iniciativa  \
0             España  sobre medidas que se están 

## Mismo proceso de iteración por la carpeta de <u>**boletines autonómicos**</u>, pero esta vez identifica y levanta solamente los archivos que hayan sido modificados o subidos después de cierta fecha. Luego, los inserta debajo de la hoja correspondiente de base de datos unificada


### ***  En el desarrollo, también incluyo otras operaciones de limpieza y transformación (los guión bajo de las fórmulas Excel, la modificación de las fechas, adaptación del timezone a Madrid, etc.)

In [ ]:
from googleapiclient.discovery import build

sheet_name = 'Boletines_unificado'
sheet_id_unificado = '1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8'
CWD = '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP (automat)/Boletines Autonómicos'
cutoff_date = pytz.timezone('Europe/Madrid').localize(datetime(2024, 10, 21, 12, 0, 0))

# Credenciales de acceso a la API de Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP (automat)/Automatización_py/test-automation-gsheets-5fa673850271.json', scope)
client = gspread.authorize(creds)


def limpiar_dataframe(dfp):
  columns_to_replace = ['Sector', 'Marco Geográfico', 'Proponente']
  for column in columns_to_replace:
      dfp[column] = dfp[column].astype(str).str.replace('_', ' ', regex=False) # Aquí soluciono lo de los guiones (_)
    # Convertir la columna 'Fecha publicación' a tipo datetime y formatearla
  dfp['Fecha de publicación'] = pd.to_datetime(dfp['Fecha de publicación'], errors='coerce').dt.strftime('%d/%m/%Y')

    # Rellenar valores nulos y convertir a lista de listas
  dfp_cleaned = dfp.fillna('')  # Rellenar los NaN con cadenas vacías
  data_cleaned = dfp_cleaned.values.tolist()  # Convertir el DataFrame a lista de listas
  return data_cleaned

def find_empty_row(worksheet):
    """Encuentra la primera fila vacía en la hoja"""
    str_list = list(filter(None, worksheet.col_values(1)))  # Obtiene la columna 1 y filtra filas vacías
    return len(str_list) + 1  # Devuelve el índice de la primera fila vacía

def obtener_sheet_id(sheet, worksheet_name):
    sheets_metadata = sheet.fetch_sheet_metadata()
    sheets = sheets_metadata['sheets']

    for s in sheets:
        if s['properties']['title'] == worksheet_name:
            return s['properties']['sheetId']

            raise Exception(f"No se encontró una hoja con el nombre: {worksheet_name}")

def aplicar_formato_fecha(sheet_id, worksheet_index, rango):
    """Aplica el formato de fecha DD/MM/YYYY en el rango de celdas especificado"""
    service = build('sheets', 'v4', credentials=creds)

    requests = [
        {
            "repeatCell": {
                "range": {
                    "sheetId": worksheet_index,  # Aquí se usa el ID correcto de la hoja
                    "startRowIndex": rango[0],  # Índice de fila de inicio (0-indexed)
                    "endRowIndex": rango[1],    # Índice de fila de fin (0-indexed)
                    "startColumnIndex": 7,
                    "endColumnIndex": 8
        },
                "cell": {
                    "userEnteredFormat": {
                        "numberFormat": {
                            "type": "DATE",
                            "pattern": "DD/MM/YYYY"
                        }
                    }
                },
                "fields": "userEnteredFormat.numberFormat"
            }
        }
                   ]
    body = {
        'requests': requests
    }

    # Ejecutar la solicitud batchUpdate para aplicar el formato de fecha
    service.spreadsheets().batchUpdate(spreadsheetId=sheet_id, body=body).execute()


def main():
    files = os.listdir(CWD)
    dataframes = []

    for file_ in files:
        file_path = os.path.join(CWD, file_)

        if os.path.isfile(file_path) and file_.endswith('.xlsx'):
            # Obtener la fecha de modificación del archivo
            # file_mod_time = datetime.fromtimestamp(os.path.getmtime(file_path))
            file_mod_time = datetime.fromtimestamp(os.path.getmtime(file_path), pytz.utc).astimezone(pytz.timezone('Europe/Madrid'))


            # Procesar solo los archivos modificados después de la fecha de corte
            if file_mod_time > cutoff_date:
                dfp = pd.read_excel(file_path, engine='openpyxl')
                dataframes.append(dfp)
                print("DataFrame creado para el archivo:", file_)
                print(dfp)

                # Limpiar el DataFrame
                data_cleaned = limpiar_dataframe(dfp)

                # Obtener el cliente de Google Sheets y abrir el archivo unificado
                client = gspread.authorize(creds)
                sheet = client.open_by_key(sheet_id_unificado)

                # Acceder a la hoja del archivo unificado
                worksheet_boletines_autonomicos = sheet.get_worksheet(0)
                sheet_id = obtener_sheet_id(sheet, worksheet_boletines_autonomicos.title)
                # Obtener el índice de la fila donde deseas insertar los datos
                indice_fila_insercion = find_empty_row(worksheet_boletines_autonomicos)
                print("Índice de fila de inserción:", indice_fila_insercion)

                # Insertar los datos limpios en la segunda hoja del archivo unificado
                worksheet_boletines_autonomicos.insert_rows(data_cleaned, indice_fila_insercion, value_input_option='USER_ENTERED')
                print("Datos insertados correctamente en la fila", indice_fila_insercion)


                start_row = indice_fila_insercion - 1
                end_row = start_row + len(data_cleaned)
                aplicar_formato_fecha(sheet_id_unificado, sheet_id, (start_row, end_row, 7, 8))

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


DataFrame creado para el archivo: 241007 Boletines Autonómicos.xlsx
      ID        Sector   Sector2  Sector3                        Subsector  \
0   2527       Empresa       NaN      NaN                              NaN   
1   2528  Alimentación  Primario      NaN                              NaN   
2   2529       Empresa  Primario      NaN                              NaN   
3   2530       Energía       NaN      NaN           Energía - Electricidad   
4   2531       Empresa       NaN      NaN                              NaN   
5   2532       Empresa       NaN      NaN  Empresa - Formación profesional   
6   2533       Empresa       NaN      NaN  Empresa - Formación profesional   
7   2534       Empresa       NaN      NaN  Empresa - Formación profesional   
8   2535      Primario       NaN      NaN             Primario - Ganadería   
9   2536         Salud       NaN      NaN                              NaN   
10  2537        Tabaco       NaN      NaN                              Na

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


DataFrame creado para el archivo: 241004 Boletines Autonómicos.xlsx
      ID         Sector   Sector2  Sector3                        Subsector  \
0   2510       Primario       NaN      NaN                              NaN   
1   2511       Primario       NaN      NaN                              NaN   
2   2512       Primario       NaN      NaN                              NaN   
3   2513       Primario       NaN      NaN                              NaN   
4   2514  Institucional       NaN      NaN                              NaN   
5   2515  Institucional       NaN      NaN                              NaN   
6   2516   Alimentación  Primario      NaN                              NaN   
7   2517        Energía       NaN      NaN           Energía - Electricidad   
8   2518        Empresa       NaN      NaN  Empresa - Formación profesional   
9   2519       Comercio       NaN      NaN                              NaN   
10  2520   Alimentación       NaN      NaN                    

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


...

DataFrame creado para el archivo: Copy of 241017 Boletines Autonómicos.xlsx
      ID          Sector         Sector2    Sector3  Subsector  \
0   1673        Primario             NaN        NaN        NaN   
1   1674           Salud  Digitalización        NaN        NaN   
2   1675   Institucional             NaN        NaN        NaN   
3   1676   Institucional             NaN        NaN        NaN   
4   1677   Institucional             NaN        NaN        NaN   
5   1678   Institucional             NaN        NaN        NaN   
6   1679   Institucional             NaN        NaN        NaN   
7   1680   Institucional             NaN        NaN        NaN   
8   1681   Institucional             NaN        NaN        NaN   
9   1682   Institucional             NaN        NaN        NaN   
10  1683  Digitalización         Empresa  Industria        NaN   
11  1684   Institucional             NaN        NaN        NaN   
12  1685         Empresa             NaN        NaN        NaN   

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


DataFrame creado para el archivo: 241018 Boletines Autonómicos.xlsx
      ID         Sector     Sector2  Sector3             Subsector  \
0   1689     Financiero         NaN      NaN                   NaN   
1   1690        Turismo         NaN      NaN                   NaN   
2   1691  Institucional         NaN      NaN                   NaN   
3   1692      Industria         NaN      NaN                   NaN   
4   1693        Empresa         NaN      NaN                   NaN   
5   1694  Institucional         NaN      NaN                   NaN   
6   1695  Institucional         NaN      NaN                   NaN   
7   1696     Innovación         NaN      NaN                   NaN   
8   1697  Institucional         NaN      NaN                   NaN   
9   1698  Institucional         NaN      NaN                   NaN   
10  1699  Institucional         NaN      NaN                   NaN   
11  1700  Institucional         NaN      NaN                   NaN   
12  1701  Institucion

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


DataFrame creado para el archivo: Copy of 241021 Boletines Autonómicos.xlsx
      ID            Sector            Sector2  Sector3  \
0   1708  Infraestructuras                NaN      NaN   
1   1709          Primario                NaN      NaN   
2   1710           Empresa           Comercio      NaN   
3   1711          Primario                NaN      NaN   
4   1712          Primario                NaN      NaN   
5   1713           Energía  Economía_Circular  Turismo   
6   1714        Innovación                NaN      NaN   
7   1715           Empresa     Digitalización      NaN   
8   1716           Empresa     Digitalización      NaN   
9   1717           Energía      Medioambiente      NaN   
10  1718            Tabaco                NaN      NaN   
11  1719             Salud                NaN      NaN   

                 Subsector Tema      Marco Geográfico Fecha de publicación  \
0                      NaN  NaN              Asturias           2024-10-21   
1   Primario

## Mismo proceso de iteración por las carpeta de <u>**agendas parlamentarias**</u>, pero esta vez identifica y levanta solamente los archivos que hayan sido modificados o subidos después de cierta fecha. Luego, los inserta debajo de la hoja correspondiente de base de datos unificada

### ***  En el desarrollo, también incluyo otras operaciones de limpieza y transformación (los guión bajo de las fórmulas Excel, la modificación de las fechas, adaptación del timezone a Madrid, etc.)

In [ ]:
from datetime import datetime

sheet_name = 'Boletines_unificado'
sheet_id_unificado = '1T8l8omO07iWkUyB2tuc6wGf_RJhsgifWUgTb6L2LPN8'
CWD = '/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP (automat)/Agendas parlamentarias'
cutoff_date = madrid_tz.localize(datetime(2024, 10, 21, 14, 35, 0))  # Fecha de corte

# Credenciales de acceso a la API de Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/Shareddrives/(02 MAD) Iniciativas - AAPP - JP (automat)/Automatización_py/test-automation-gsheets-5fa673850271.json', scope)
client = gspread.authorize(creds)
gc = gspread.authorize(creds)

def limpiar_dataframe(dfp):
    columns_to_replace = ['Sector', 'Marco geográfico', 'Proponente']
    for column in columns_to_replace:
      dfp[column] = dfp[column].astype(str).str.replace('_', ' ', regex=False) # Aquí soluciono lo de los guiones (_)
    dfp['Fecha convocada'] = pd.to_datetime(dfp['Fecha convocada'], errors = 'coerce').dt.strftime('%d/%m/%Y')
    # Reemplazar NaN con una cadena vacía
    dfp_cleaned = dfp.fillna('')
    # Convertir DataFrame a lista de listas
    data_cleaned = dfp_cleaned.values.tolist()
    return data_cleaned


def find_empty_row(worksheet):
    """Encuentra la primera fila vacía en la hoja"""
    str_list = list(filter(None, worksheet.col_values(1)))  # Obtiene la columna 1 y filtra filas vacías
    return len(str_list) + 1  # Devuelve el índice de la primera fila vacía

def obtener_sheet_id(sheet, worksheet_name):
    sheets_metadata = sheet.fetch_sheet_metadata()
    sheets = sheets_metadata['sheets']

    for s in sheets:
        if s['properties']['title'] == worksheet_name:
            return s['properties']['sheetId']

            raise Exception(f"No se encontró una hoja con el nombre: {worksheet_name}")

def aplicar_formato_fecha(sheet_id, worksheet_index, rango):
    """Aplica el formato de fecha DD/MM/YYYY en el rango de celdas especificado"""
    service = build('sheets', 'v4', credentials=creds)

    requests = [

         {
             "repeatCell": {
                "range": {
                    "sheetId": worksheet_index,  # Aquí se usa el ID correcto de la hoja
                    "startRowIndex": rango[0], #índice de fila de inicio (0-indexed)
                    "endRowIndex": rango[1], #índice de fila de fin (0-indexed)
                    "startColumnIndex": 7,
                    "endColumnIndex": 8

                },
                "cell": {
                    "userEnteredFormat": {
                        "numberFormat": {
                            "type": "DATE",
                            "pattern": "DD/MM/YYYY"
                        }
                    }
                },
                "fields": "userEnteredFormat.numberFormat"
             }
         }
    ]

    body = {
        'requests': requests
    }

    # Ejecutar la solicitud batchUpdate para aplicar el formato de fecha
    service.spreadsheets().batchUpdate(spreadsheetId=sheet_id, body=body).execute()


def main():
    files = os.listdir(CWD)
    dataframes = []
    for file_ in files:
        file_path = os.path.join(CWD, file_)
        if os.path.isfile(file_path) and file_.endswith('.xlsx'):
          file_mod_time = datetime.fromtimestamp(os.path.getmtime(file_path), pytz.utc).astimezone(madrid_tz)


# Procesar solamente los archivos modificados depsués de la fecha de corte
          if file_mod_time > cutoff_date:
            dfp = pd.read_excel(file_path, engine= 'openpyxl')
            dataframes.append(dfp)
            print("DataFrame creado para el archivo:", file_)
            print(dfp)

            # Limpiar el DataFrame
            data_cleaned = limpiar_dataframe(dfp)

            # Obtener el cliente de Google Sheets y abrir el archivo unificado
            client = gspread.authorize(creds)
            sheet = client.open_by_key(sheet_id_unificado)

            # Acceder a la segunda hoja del archivo unificado
            worksheet_agendas_parlamentarias = sheet.get_worksheet(2)
            sheet_id = obtener_sheet_id(sheet, worksheet_agendas_parlamentarias.title)


            # Obtener el índice de la fila donde deseas insertar los datos
            indice_fila_insercion = find_empty_row(worksheet_agendas_parlamentarias)
            print("Índice de fila de inserción:", indice_fila_insercion)

            # Insertar los datos limpios en la segunda hoja del archivo unificado
            worksheet_agendas_parlamentarias.insert_rows(data_cleaned, indice_fila_insercion, value_input_option = 'USER_ENTERED')
            print("Datos insertados correctamente en la fila", indice_fila_insercion)

            start_row = indice_fila_insercion - 1
            end_row = start_row + len(data_cleaned)
            aplicar_formato_fecha(sheet_id_unificado, sheet_id, (start_row, end_row, 7, 8))

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


DataFrame creado para el archivo: 240923_Agendas Parlamentarias (1).xlsx
      ID         Sector     Sector2 Subsector              Tema Legislatura  \
0   2732        Turismo         NaN       NaN               NaN          XI   
1   2733        Turismo    Vivienda       NaN               NaN          XI   
2   2734        Turismo         NaN       NaN               NaN          XI   
3   2735        Turismo         NaN       NaN    Sostenibilidad          XI   
4   2736          Salud         NaN       NaN  Listas de Espera          XI   
..   ...            ...         ...       ...               ...         ...   
62  2794        Energía         NaN       NaN               NaN          XI   
63  2795  Institucional   Movilidad       NaN               NaN        XIII   
64  2796  Institucional  Innovación       NaN               NaN        XIII   
65  2797  Institucional     Turismo       NaN               NaN        XIII   
66  2798  Institucional    Primario       NaN             

# **Et... voilà!**